In [1]:
import PyLidar3
import os
import re
import csv
import math
import numpy as np
from pynq import allocate
from pynq import Overlay
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import time

ol = Overlay('/home/xilinx/pynq/overlays/clusterOp2/clusterOp2.bit')

angle = np.arange(0, 360)
distance = []


#port = "/dev/ttyUSB0" #linux
#Obj = PyLidar3.YdLidarX4(port, 12000) 

# To use with csv file instead of LIDAR sensor, make sure 
# there is a folder called data in the same working directory
# and there is a CSV file called "data.csv" in that folder

with open('./data/data.csv') as lidarData:
    lidar_reader = csv.reader(lidarData)
        
    for point in lidar_reader:
        distance.append(np.uint32(point[1]))

In [2]:
#if(Obj.Connect()):
#    print(Obj.GetDeviceInfo())
#else:
#    print("Error connecting to device")

In [3]:
#gen = Obj.StartScanning()
#data = next(gen)
#for points in data:
#    distance.append(data[points])
#print(distance)
#Obj.StopScanning()

In [4]:
#Obj.Disconnect()

In [5]:
#Processes the raw data into objects, removes noise
def clusterOp(angle, distance, D_th = 100, minGroupSize = 4):
    objects = []
    grouped = set()

    for point in range(0,360):

        #If point has already been grouped move onto the next point
        if point in grouped:
            continue

        cluster = set()
        cluster.add(point)

        #Test in counter clockwise direction
        for i in range(0,360):
            dTheta = angle[i] - angle[point]
            d1 = distance[point]
            d2 = distance[i]
            d3 = math.sqrt(d1**2 + d2**2 - 2*d1*d2*math.cos(dTheta))

            
            if d3 < D_th:
                cluster.add(i)
                grouped.add(i)
                point = i

        if len(cluster) >= minGroupSize:
            objects.append(cluster)

    return objects

In [6]:
ol.clusterOp2_0.register_map
dma = ol.axi_dma
input_buffer = allocate(shape=(360,), dtype = np.int32)
np.copyto(input_buffer, distance)
output_buffer = allocate(shape=(380,), dtype=np.int32)
#print(input_buffer)

In [7]:
start = time.time()

CONTROL_REGISTER = 0x0
ol.clusterOp2_0.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

dma.sendchannel.transfer(input_buffer)
dma.sendchannel.wait()
dma.recvchannel.transfer(output_buffer)

clusterOp_data = output_buffer

del input_buffer, output_buffer

cluster_n = []  # initialize empty list for first cluster
clusters = []  # initialize list to hold all clusters

for value in clusterOp_data:
    if value == 720:
        if cluster_n:  # check if cluster_n already has values
            clusters.append(cluster_n)  # if so, add it to clusters
            cluster_n = []  # reset cluster_n to empty list
    else:
        cluster_n.append(value)  # add value to current cluster

if cluster_n:  # handle the last cluster after the loop ends
    clusters.append(cluster_n)
    
end = time.time()

print(end-start)

print(clusters)  # display all clusters
    

0.010261774063110352
[[1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 320, 321, 334, 335, 338, 340, 342, 343, 344, 355, 357, 358, 359], [151, 160, 161, 162, 163, 164, 165, 174, 175, 189, 199, 200, 201, 202, 204, 205], [206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 276, 277, 278, 339, 356], [245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271], [273, 274, 275, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [8]:
start = time.time()
clusters = clusterOp(angle, distance)
end = time.time()
print(end-start)
print(clusters)  # display all clusters


28.064834117889404
[{11, 279, 280, 281, 282, 283, 157, 158, 159, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 190, 191, 192, 193, 194, 195, 196, 197, 198, 318, 319, 73, 341, 217, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 243}, {226, 163, 12, 207}, {226, 163, 12, 207, 213}, {218, 212, 149, 231}, {226, 163, 232, 12, 207}, {209, 234, 228, 165}, {226, 163, 12, 207, 276}, {205, 224, 4, 293}]
